## Weather API

In [363]:
from datetime import datetime
import requests
import json
import config
import pandas as pd

In [364]:
weather_api_key = config.api_key

weather_test = requests.get("http://api.weatherapi.com/v1/current.json", params={"key": weather_api_key, "q": "London"})
weather_json_test = json.loads(weather_test.text)
print(weather_json_test.get("current").get("last_updated"))

2024-04-09 15:45


In [365]:
def get_weather_data(location, wtype, interval, apikey):
    columns = ["date", "max_temp", "min_temp", "avg_temp", "total_precipitation", "type"]
    loc_lower = location.lower()
    wtype_lower = wtype.lower()

    params = {"key": apikey,
              "q": loc_lower,
              }

    if wtype_lower == "forecast":
        params["days"] = interval
        weather = requests.get(f"https://api.weatherapi.com/v1/{wtype_lower}.json", params=params)
        weather_json = json.loads(weather.text)

        weather_df = convert_weather_data_to_df(weather_json, columns)
    elif wtype_lower == "history":
        try:
            datetime.strptime(interval, '%Y-%m-%d')
            if interval < "2010-01-01":
                raise ValueError("Date must be 2010-01-01 or later")
        except ValueError:
            raise ValueError("Date must have the format yyyy-MM-dd")
        params["dt"] = interval
    
        weather = requests.get(f"https://api.weatherapi.com/v1/{wtype_lower}.json", params=params)
        weather_json = json.loads(weather.text)

        weather_df = convert_weather_data_to_df(weather_json, columns)
    else:
        raise ValueError("Wrong weather type")

    return weather_df

In [366]:
def convert_weather_data_to_df(weather_json, columns):
    if 'forecast' in weather_json and 'forecastday' in weather_json['forecast']:
        history_data = weather_json['forecast']['forecastday']
        weather_data = []
        for history in history_data:
            date = history['date']
            max_temp = history['day']['maxtemp_c']
            min_temp = history['day']['mintemp_c']
            avg_temp = history['day']['avgtemp_c']
            total_precipitation = history['day']['totalprecip_mm']
            weather_type = history['day']['condition']['text']
            weather_data.append([date, max_temp, min_temp, avg_temp, total_precipitation, weather_type])

        return pd.DataFrame(weather_data, columns=columns)
    else:
        print("No forecast data found.")

In [367]:
print(get_weather_data("LONDON", "FORECAST", "3", weather_api_key))

         date  max_temp  min_temp  avg_temp  total_precipitation  \
0  2024-04-09      11.5       6.8       8.8                 0.59   
1  2024-04-10      14.0       5.0       9.3                 0.01   
2  2024-04-11      15.1      11.5      13.4                 0.04   

                 type  
0  Patchy rain nearby  
1           Overcast   
2           Overcast   


In [368]:
print(get_weather_data("London", "History", "2024-04-03", weather_api_key))

         date  max_temp  min_temp  avg_temp  total_precipitation  \
0  2024-04-03      14.1       9.5      11.4                 2.94   

                type  
0  Light rain shower  
